In [1]:
from os.path import join as pathjoin
from data_processing import *
from interpretation import *
from models import *
from training import *

Let's download all the data.

In [2]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'

In [3]:
!ls /home/mlepekhin/data/min_gpt_bpe

en		ru_topic
en_topic	ru_topic_big_sep_generators
ru		ru_topic_big_sep_generators.csv
ru_lemma.csv	ru_train_topic_big_sep_generators.csv
ru_one_dataset	single_ru_topic


In [4]:
import pandas as pd

df1 = pd.read_csv(pathjoin(DATA_DIR, 'min_gpt_bpe/ru_train_topic_big_sep_generators.csv'))
#df2 = pd.read_csv(pathjoin(DATA_DIR, 'ru_train'))
#df1.append(df2).to_csv(pathjoin(DATA_DIR, "ru_adversarial_train"))
topic_list = np.unique(df1.topic.values)
print(topic_list)
for topic in topic_list:
    df1[df1.topic == topic].to_csv(pathjoin(DATA_DIR, f"ru_adversarial_{topic}.csv"))

['arhitecture' 'business' 'crime' 'education' 'games' 'literature' 'music'
 'politics' 'sport' 'travel']


In [5]:
transformer_model = 'DeepPavlov/rubert-base-cased'
MAX_TOKENS = 512

In [6]:
for topic in topic_list[4:]:
    MODEL_ID = f'allennlp_rubert_from_topic_generated_{topic}'
    CHECKPOINTS_DIR = pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints')
    BEST_MODEL = pathjoin(CHECKPOINTS_DIR, 'best.th')

    train_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)
    val_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)

    train_data, dev_data = read_data(
        pathjoin(DATA_DIR, pathjoin(DATA_DIR, f"ru_adversarial_{topic}.csv")), 
        pathjoin(DATA_DIR, "ru_test"),
        train_dataset_reader, 
        val_dataset_reader
    )

    vocab = build_vocab(train_data + dev_data)
    train_data.index_with(vocab)
    dev_data.index_with(vocab)
    
    model = build_transformer_model(vocab, transformer_model)
    if torch.cuda.is_available():
        cuda_device = 0
        model = model.cuda(cuda_device)
    else:
        cuda_device = -1
    print(cuda_device)
    
    train_loader, dev_loader = build_data_loaders(train_data, dev_data)
    # You obviously won't want to create a temporary file for your training
    # results, but for execution in binder for this course, we need to do this.
    !rm -rf {CHECKPOINTS_DIR}
    !mkdir -p {CHECKPOINTS_DIR}
    trainer = build_classifier_trainer(
        model,
        pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints'),
        train_loader,
        dev_loader,
        5,  
        cuda_device=cuda_device
    )
    print("Starting training")
    trainer.train()
    print("Finished training")
    
    !rm "{CHECKPOINTS_DIR}"/*.json "{CHECKPOINTS_DIR}"/model* "{CHECKPOINTS_DIR}"/training*
    model.vocab.save_to_files(pathjoin(MODELS_DIR, MODEL_ID, 'vocab'))
    !ls -lh "{CHECKPOINTS_DIR}"

Reading data
<class 'data_processing.ClassificationDatasetReader'> /home/mlepekhin/data/ru_adversarial_games.csv



Building the vocabulary



Building the model
0


You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


Starting training


...


Building the vocabulary



Building the model
0


You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


Starting training



Finished training
total 681M
-rw-rw-r-- 1 mlepekhin mlepekhin 681M окт  7 23:28 best.th
drwxrwxr-x 4 mlepekhin mlepekhin 4,0K окт  7 23:25 log
